In [1]:
# https://blog.finxter.com/how-to-create-an-interactive-web-application-using-jupyter-notebook/
# https://github.com/binder-examples/voila
# https://github.com/voila-dashboards/voila/tree/stable/notebooks
# https://voila-gallery.org/
from pyproj import Transformer, CRS
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'colab'  # "browser" for pycharm, "notebook" for jupyter
#import kaleido  # to render graphs in plotly as images
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from io import StringIO
import csv
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML # Start writing code here...
# https://github.com/jupyter-widgets/ipywidgets/issues/3445

In [2]:
DATA_URL = 'https://gist.githubusercontent.com/chriddyp/cb5392c35661370d95f300086accea51/raw/8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/indicators.csv'
EXPLANATION = """\
<div class="app-sidebar">
<p><em>Compare different development indicators.</em><p>

<p>Select what indicators to plot in the dropdowns, and use the slider
to sub-select a fraction of years to include in the plot.</p>

<p>Data and idea copied from the <a href="https://dash.plot.ly/getting-started-part-2">
Plotly Dash documentation</a>.</p>

<p>This example demonstrates combining matplotlib with Jupyter widgets. For more interactive plots,
consider using <a href="https://github.com/bloomberg/bqplot">bqplot</a>.
</div>
"""

In [3]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [12]:
class App:
    
    def __init__(self, df):
        self._df = df
        available_indicators = self._df['Indicator Name'].unique()
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, 0)
        self._y_dropdown = self._create_indicator_dropdown(available_indicators, 1)  
        self._upload = self._create_upload_button()
        self._tbl = self._create_dataframe_box()
        self._plot_container = widgets.Output()
        self._year_slider, year_slider_box = self._create_year_slider(
            min(df['Year']), max(df['Year'])
        )
        _app_container = widgets.VBox([
            widgets.HBox([self._x_dropdown, self._y_dropdown]),
            self._upload,
            self._tbl,
            self._data_container,
            self._plot_container,
            year_slider_box
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>Development indicators</h1>'
                    '<h2 class="app-subtitle"><a href="https://github.com/pbugnion/voila-gallery/blob/master/country-indicators/index.ipynb">Link to code</a></h2>'
                ), 
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container, 
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()
        
    @classmethod
    def from_url(cls, url):
        df = pd.read_csv(url)
        return cls(df)

    # get filename from FileUpload dict
    def get_name(file_upload):
        if len(file_upload) == 0:
            return ''
        return next(iter(file_upload.values()))['metadata']['name']
    
    # display dataframe
    def setup_ui(df):
        out = widgets.Output()
        with out:
            display(df)
        return out

    def _create_upload_button(self):
        upload = widgets.FileUpload(accept='.xlsx', multiple=False)
        upload.observe(self.show_it, names=['value'])
        return upload
    
    def _create_dataframe_box(self):
        df_box_label = widgets.Label('Data: ')
        if self._upload._counter != 0:
            tbl = pd.DataFrame(data={'Element name': 'P1', 'Easting/Rechtswert': 0,\
                                    'Northing/Hochwert': 0}, index=[0])
        else:
            input_file = list(self._upload.value.values())[0]
            content = input_file['content']  # equal to upload.data[0]
            if uploaded_filename.find('.xls') != -1:
                self._tbl = pd.read_excel(content, header=0, engine='openpyxl')
            elif uploaded_filename.find('.csv') != -1:
                content = StringIO(content.decode('utf-8'))
                sniffer = csv.Sniffer()
                sniffer.preferred = [';', ',']
                dialect = sniffer.sniff(content)
                self._tbl = pd.read_csv(content, delimiter=dialect.delimiter, header=0)
            elif uploaded_filename.find('.txt') != -1:
                content = StringIO(content.decode('utf-8'))
                self._tbl = pd.read_csv(content, sep=" ", header=None)
        df_box = widgets.VBox([df_box_label, year_slider])
        year_slider.observe(self._on_change, names=['value'])
        return df_box_label, df_box
        
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown

    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Year range: ')
        year_slider = widgets.IntRangeSlider(
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px')
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box
    
    def _create_plot(self, x_indicator, y_indicator, year_range):
        df = self._df[self._df['Year'].between(*year_range)]
        xs = df[df['Indicator Name'] == x_indicator]['Value']
        ys = df[df['Indicator Name'] == y_indicator]['Value']
        plt.figure(figsize=(10, 8))
        plt.xlabel(x_indicator, size=16)
        plt.ylabel(y_indicator, size=16)
        plt.gca().tick_params(axis='both', which='major', labelsize=16)
        plt.plot(xs, ys, 'o', alpha=0.3)

    def show_it(self):
        if self._upload._counter != 0:  #self._upload.value != {}:
            #out = widgets.Output()
            print('Uploaded ' + str((list(self._upload['new'].keys())[-1])))
            print(self._upload)
            input_file = list(self._upload.value.values())[0]
            content = input_file['content']  # equal to upload.data[0]
            if uploaded_filename.find('.xls') != -1:
                self._tbl = pd.read_excel(content, header=0, engine='openpyxl')
            elif uploaded_filename.find('.csv') != -1:
                content = StringIO(content.decode('utf-8'))
                sniffer = csv.Sniffer()
                sniffer.preferred = [';', ',']
                dialect = sniffer.sniff(content)
                self._tbl = pd.read_csv(content, delimiter=dialect.delimiter, header=0)
            elif uploaded_filename.find('.txt') != -1:
                content = StringIO(content.decode('utf-8'))
                self._tbl = pd.read_csv(content, sep=" ", header=None)
            #with out:
                #display(tbl)
            print('done')
            return self._tbl
        else:
            return ''

    def _on_change(self, _):
        self._update_app()

    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        year_range = self._year_slider.value
        self.show_it()
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(x_indicator, y_indicator, year_range)
            plt.show()

In [13]:
app = App.from_url(DATA_URL)

app.container

IndexError: list index out of range

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a82d000-c111-4d9b-b2f0-364d0f452677' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>